# Student

In [45]:
import numpy as np

# ==========================================================================================
class Student:
    def __init__(self, similar=[], learn_rate=.1, forget_rate=.1, sim_rate=.5):
        self.skills = {}
        self.sim_subjs = similar
        self.lr = learn_rate
        self.fr = forget_rate
        self.sr = sim_rate

    def clear(self):
        for s in self.skills.keys():
            self.skills[s] = 0

    def learn(self, subj):
        # update subj's skill
        v = self.skills.get(subj, 0)
        self.skills[subj] = min(v+self.lr, 1) 
        # update similar skills
        for sim in self.sim_subjs.get(subj, []):
            v = self.skills.get(sim, 0)
            self.skills[sim] = min(v+self.lr*self.sr, 1) 

    def forget(self):
        for s,v in self.skills.items():
            self.skills[s] = v*(1-self.fr)
            
    def __repr__(self):
        return "%s(%r)" % (self.__class__, self.__dict__)

# ==========================================================================================
class BaseTeacher:
    def __init__(self, subjs):
        self.subjs = subjs

    def teach(self, student):
        if type(student) is list:
            for s in student:
                self.teach(s)
        self._do_teach(student)
    
    def _do_teach(self, student):
        raise NotImplementedError

        
class Teacher(BaseTeacher):
    def _do_teach(self, student):
        subj = np.random.choice(self.subjs)
        student.learn(subj)
        #print(s)


In [46]:
def make_add_subjects(num):
    return ["%d+%d=%d" % (i+1,j+1,i+j+2) for i in range(num) for j in range(num)]

def split_subjects(subjs, num):
    g = [[] for _ in range(num)]
    for s in subjs:
        i = np.random.randint(num)
        g[i].append(s)
    return g
            
def make_similar(subjs, num):
    groups = split_subjects(subjs, num)
    sims = {}
    for g in groups:
        for s in g:
            sims[s] = [sim for sim in g if sim!=s]
    return sims

def evaluate(student, subjs):
    v = 0
    for s in subjs:
        v += student.skills.get(s, 0)
    return v / len(subjs)

In [48]:
np.random.seed(1969)

subjs = make_add_subjects(3)
sim_subjs = make_similar(subjs, 6)
print("subjs:", subjs, '\n')
print("sim_subjs:", sim_subjs, '\n')

subjs: ['1+1=2', '1+2=3', '1+3=4', '2+1=3', '2+2=4', '2+3=5', '3+1=4', '3+2=5', '3+3=6'] 

sim_subjs: {'2+1=3': [], '1+1=2': ['2+3=5'], '2+3=5': ['1+1=2'], '1+2=3': ['3+1=4', '3+3=6'], '3+1=4': ['1+2=3', '3+3=6'], '3+3=6': ['1+2=3', '3+1=4'], '1+3=4': ['3+2=5'], '3+2=5': ['1+3=4'], '2+2=4': []} 



In [65]:
t = Teacher(subjs)
s = Student(sim_subjs, learn_rate=1.)

print(evaluate(s, subjs))
print(s.skills, '\n')

for _ in range(10):
    t.teach(s)
    
print(evaluate(s, subjs))
print(s.skills, '\n')
    

0.0
{} 

0.6111111111111112
{'1+2=3': 1, '3+1=4': 1, '3+3=6': 1, '2+2=4': 1.0, '3+2=5': 1.0, '1+3=4': 0.5} 



In [4]:
s = Student(sim_subjs, learn_rate=1.)
print(evaluate(s, subjs))
print(s.skills, '\n')

s.learn('1+2=3')
print(evaluate(s, subjs))
print(s.skills, '\n')

s.clear()

s.learn('1+1=2')
print(evaluate(s, subjs))
print(s.skills, '\n')

0.0
{} 

0.2222222222222222
{'1+2=3': 1.0, '3+1=4': 0.5, '3+3=6': 0.5} 

0.16666666666666666
{'1+2=3': 0, '3+1=4': 0, '3+3=6': 0, '1+1=2': 1.0, '2+3=5': 0.5} 

